In [ ]:
#필요라이브러리 설치
!pip install gluonnlp pandas tqdm   
!pip install mxnet
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-5sedm2k8/kobert-tokenizer_7fee7c6dcc4744f4a1bc63bff9ada3de
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-5sedm2k8/kobert-tokenizer_7fee7c6dcc4744f4a1bc63bff9ada3de


In [ ]:
#kobert 다운
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-93y0e2yu
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-93y0e2yu


In [ ]:
def get_kobert_model(model_path,vocab_file,ctx="cpu"):
  bertmodel = BertModel.from_pretrained(model_path)
  device = torch.device(ctx)
  bertmodel.to(device)
  bertmodel.eval()
  vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(vocab_file,padding_token='[PAD]')
  return bertmodel, vocab_b_obj

In [ ]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1")
model = AutoModel.from_pretrained("skt/kobert-base-v1")

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

#GPU 사용 시
device = torch.device("cuda:0")

In [ ]:
#from kobert.pytorch_kobert import get_kobert_model
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1',tokenizer.vocab_file)

In [ ]:
data = pd.read_csv('13department.csv')

In [ ]:
data

,Unnamed: 0,제목,내용,담당부서
0,0,인접지 공사에 따른 피해조치 요구 및 공사중지 요청,제목: 인접지 공사에 따른 피해조치 요구 및 공사중지 요청 수신인: 동대문구청장 참...,도시관리국 건축과
1,1,쓰레기 수거 용역업체에서 금전을 요구합니다.,동대문구 장안동에서 자영업을 하고 있는 개인 사업자 입니다. 쓰레기 수거 용역업체에...,안전생활국 청소행정과
2,4,경동시장 차광막 공사로 인도에 쌓인 불법 적치물 창고를 만들어주는 건가요?,이전 민원 시 불법적치물 과태료를 부과해서 인도의 적치된 물건들 치울 계획이라는 답...,기획재정국 경제진흥과
3,6,해충약 신청합니다,작년여름 7ㅡ10월까지 당주소 1층화단과 옆 우민에뜨와르아파트사이 나무등등에서 송충...,도시관리국 공원녹지과
4,7,서울시립대로12길 볼라드 설치 요청,담당자 분과 통화한거와 같이 만물마트 부근 만연한 불법 주정차로 인하여 통행권이 확...,건설교통국 교통행정과
...,...,...,...,...
1125,1375,장안2동마로니에 놀이터 미끄럼틀 너무 위험해,작년에 새로 정비된 마로니에 어린이 공원에는미끄럼틀이 있습니다 그중 큰것에 구름다리...,도시관리국 공원녹지과
1126,1377,동대문구청 주.정차관 단속 벌금 과연 제대로 국고로 납부되는가?,안녕하십니까.\n\n강남구에 살고 있는 사람이지만 동대문 구청의 행정처리에 의문을 ...,건설교통국 주차행정과
1127,1379,용신1교 환경 개선 요청드립니다.,"용신1교에 방치되어 있는 방범용 컨테이너 및 짐차, 쓰레기차에 대한 조치 부탁드립니...",건설교통국 건설관리과
1128,1381,거주자 우선주차장 주차료 너무 과합니다.,거주자 우선주차장 주차료 너무 과하다\n\n1.무슨근거로 주차료를 그렇게 산정한건지...,건설교통국 주차행정과


In [ ]:
data.loc[(data['담당부서'] == "도시관리국 건축과"), '담당부서'] = 0  
data.loc[(data['담당부서'] == "기획재정국 경제진흥과"), '담당부서'] = 1  
data.loc[(data['담당부서'] == "도시관리국 공원녹지과"), '담당부서'] = 2  
data.loc[(data['담당부서'] == "건설교통국 주차행정과"), '담당부서'] = 3  
data.loc[(data['담당부서'] == "건설교통국 건설관리과"), '담당부서'] = 4  
data.loc[(data['담당부서'] == "안전생활국 청소행정과"), '담당부서'] = 5  
data.loc[(data['담당부서'] == "건설교통국 교통행정과"), '담당부서'] = 6 
data.loc[(data['담당부서'] == "안전생활국 맑은환경과"), '담당부서'] = 7 
data.loc[(data['담당부서'] == "도시관리국 도시계획과"), '담당부서'] = 8 
data.loc[(data['담당부서'] == "보건소 지역보건과"), '담당부서'] = 9 
data.loc[(data['담당부서'] == "건설교통국 도로과"), '담당부서'] = 10 
data.loc[(data['담당부서'] == "도시관리국 주택과"), '담당부서'] = 11
data.loc[(data['담당부서'] == "건설교통국 치수과"), '담당부서'] = 12 


In [ ]:
data_list = []
for ques, label in zip(data['내용'], data['담당부서'])  :
    data = []   
    data.append(ques)
    data.append(str(label))

    data_list.append(data)

In [ ]:
max_len = 250
batch_size = 32
warmup_ratio = 0.1
num_epochs = 10  
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=34)

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
tok=tokenizer.tokenize
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test,0, 1, tok, vocab,  max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=13,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
 
#optimizer와 schedule
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [ ]:
train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/29 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 2.552462577819824 train acc 0.03125
epoch 1 train acc 0.12284482758620689


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 9 test acc 0.73828125


  0%|          | 0/29 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.3397164046764374 train acc 0.96875
epoch 10 train acc 0.9644396551724138


  0%|          | 0/8 [00:00<?, ?it/s]

epoch 10 test acc 0.7265625


In [ ]:
#예측
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("도시관리국 건축과")
            elif np.argmax(logits) == 1:
                test_eval.append("기획재정국 경제진흥과")
            elif np.argmax(logits) == 2:
                test_eval.append("도시관리국 공원녹지과")
            elif np.argmax(logits) == 3:
                test_eval.append("건설교통국 주차행정과")
            elif np.argmax(logits) == 4:
                test_eval.append("건설교통국 건설관리과")
            elif np.argmax(logits) == 5:
                test_eval.append("안전생활국 청소행정과")
            elif np.argmax(logits) == 6:
                test_eval.append("건설교통국 교통행정과")
            elif np.argmax(logits) == 7:
                test_eval.append("안전생활국 맑은환경과")
            elif np.argmax(logits) == 8:
                test_eval.append("도시관리국 도시계획과")
            elif np.argmax(logits) == 9:
                test_eval.append("보건소 지역보건과")
            elif np.argmax(logits) == 10:
                test_eval.append("건설교통국 도로과")
            elif np.argmax(logits) == 11:
                test_eval.append("도시관리국 주택과")
            elif np.argmax(logits) == 12:
                test_eval.append("건설교통국 치수과")
    print(">> 입력하신 민원은 " + test_eval[0] + " 로 전달되었습니다.")

#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 장안2동마로니에 놀이터 미끄럼틀 너무 위험해


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>> 입력하신 민원은 도시관리국 공원녹지과 로 전달되었습니다.


하고싶은 말을 입력해주세요 : 0
